In [21]:
from poisson_fem.mesh import PoissonFEM
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc

In [22]:
m = 8
mesh = PoissonFEM.RectangularMesh(np.ones(m)/m)
# mesh.plot()

In [23]:
def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps
def essBoundaryFun(x):
    return 1.0
mesh.setEssentialBoundary(origin, essBoundaryFun)

In [24]:
def domainBoundary(x):
    # unit square
    return np.abs(x[0]) < np.finfo(float).eps or np.abs(x[1]) < np.finfo(float).eps or \
            np.abs(x[0]) > 1.0 - np.finfo(float).eps or np.abs(x[1]) > 1.0 - np.finfo(float).eps
mesh.setNaturalBoundary(domainBoundary)

In [25]:
#Define boundary flux field
def flux(x):
    a = np.array([1, 2, 3])
    q = np.array([a[0] + a[2]*x[1], a[1] + a[2]*x[0]])
    return q

In [26]:
rs = PoissonFEM.RightHandSide(mesh)

In [27]:
rs.setNaturalRHS(mesh, flux)

In [28]:
print('naturalRHS = ', rs.naturalRHS)

naturalRHS =  [-0.296875 -0.34375  -0.390625 -0.4375   -0.484375 -0.53125  -0.578125
 -0.234375 -0.171875  0.        0.        0.        0.        0.
  0.        0.        0.171875 -0.21875   0.        0.        0.
  0.        0.        0.        0.        0.21875  -0.265625  0.
  0.        0.        0.        0.        0.        0.        0.265625
 -0.3125    0.        0.        0.        0.        0.        0.
  0.        0.3125   -0.359375  0.        0.        0.        0.
  0.        0.        0.        0.359375 -0.40625   0.        0.
  0.        0.        0.        0.        0.        0.40625  -0.453125
  0.        0.        0.        0.        0.        0.        0.
  0.453125 -0.109375  0.296875  0.34375   0.390625  0.4375    0.484375
  0.53125   0.578125  0.546875]


In [29]:
funSpace = PoissonFEM.FunctionSpace(mesh)

In [30]:
K = PoissonFEM.StiffnessMatrix(mesh, funSpace)

In [31]:
rs.setRhsStencil(mesh, K)

In [32]:
x = np.arange(5.0, m**2 + 5.0)
rs.assemble(x)
diff = rs.naturalRHS - rs.rhs_np
print('diff = ', diff)

diff =  [-0.83333333  0.          0.          0.          0.          0.
  0.          0.         -0.83333333 -1.66666667  0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.        ]


In [33]:
# Precomputations
# Kvec = PETSc.Vec().createSeq(mesh.nEq**2)
x = PETSc.Vec().createSeq(mesh.nEq)
rhs = PETSc.Vec().createSeq(mesh.nEq)

In [34]:
# Set up solver
ksp = PETSc.KSP().create()
ksp.setType('preonly')
pc = ksp.getPC()
pc.setType('cholesky')
ksp.setFromOptions() #???

In [35]:
# for scipy comparison
Kes = K.globStiffStencil.getValuesCSR()
Kes = sps.csr_matrix((Kes[2], Kes[1], Kes[0]))

In [36]:
N = 1e2

In [37]:
# r1 = range(mesh.nCells)
r2 = range(mesh.nEq)
lmbda_np = np.ones(mesh.nCells)
rhs_np = np.ones(mesh.nEq)
start = time.time()
for n in range(int(N)):
    # Stiffness matrix assembly, lmbda = [1, 1, ..., 1]
    rhs.setValues(r2, rhs_np)
    K.assemble(lmbda_np)

    # solving
    ksp.setOperators(K.matrix)
    ksp.solve(rhs, x)
petsc_time = (time.time() - start)/N
print('PETSc time = ', petsc_time)

PETSc time =  6.815671920776367e-05


In [38]:
start = time.time()
for n in range(int(N)):
    Kvecs = Kes @ np.ones(mesh.nCells)
    K1 = sps.csr_matrix((Kvecs[K.vec_nonzero], K.indices, K.indptr))
    x1 = lg.spsolve(K1, np.ones(mesh.nEq))
scipy_time = (time.time() - start)/N
print('scipy time = ', scipy_time)

scipy time =  0.00030731678009033204


In [39]:
diff = np.linalg.norm(x.array - x1)/np.linalg.norm(x1)
print('result difference = ', diff)
print('PETSc speedup = ', scipy_time/petsc_time)

result difference =  1.134057704558057e-14
PETSc speedup =  4.508972609927589


In [40]:
mesh.cells[0].containsEssentialVertex

True